In [1]:
import pandas as pd
import numpy as np
import datetime
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

empty_matrix = pd.read_hdf('..//..//data//model//model_data_12H.h5')
encampments = pd.read_csv('..//..//data//sf_open//311_Cases.csv')
blocks = pd.read_csv('..//..//data//census//tl_2010_06075_tabblock10.csv')
block_neigh = pd.read_csv('..//..//data//created//block_neigh.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
neighs = [
    
    'Tenderloin',
    'South of Market',
    'Downtown',
    'Inner Mission',
    'Van Ness/Civic Center',
    'Duboce Triangle',
    'Haight Ashbury',
    'Miraloma Park',
    'Financial District/Barbary Coast',
    'Silver Terrace' 
]

good_blocks = block_neigh[block_neigh['nbrhood_name'].isin(neighs)]['block_fips'].unique()

blocks = blocks[blocks['GEOID10'].isin(good_blocks)]

In [3]:
# get encampments reports
encampments = encampments[encampments['Request Type']=='Encampment Reports']
encampments = encampments[encampments['Latitude']!=0]

encampments['Opened_rnd'] = pd.to_datetime(encampments['Opened'])
encampments = encampments[encampments['Opened_rnd']>'2016-05-01']

In [6]:
# make polygons for blocks
def make_poly(lon, lat):
    radius = Point(lon,lat).buffer(0.01)
    poly = Polygon(radius.exterior.coords)
    return poly

blocks['poly'] = blocks.apply(lambda row: make_poly(row['INTPTLON10'],row['INTPTLAT10']), axis=1)
blocks_poly = blocks.loc[:,[
    
    'GEOID10',
    'poly'
]]

In [7]:
encamp_matrix = empty_matrix.merge(blocks_poly, left_on='block_fips', right_on='GEOID10')
encamp_matrix['Opened_rnd'] = pd.to_datetime(encamp_matrix['Opened_rnd'],format='%Y-%m-%d %H:%M:%S')

In [10]:
empty_matrix

,Opened_rnd,block_fips,CaseID,poop
0,2018-04-01 00:00:00,60750101001000,NaN,0
1,2018-04-01 12:00:00,60750101001000,NaN,0
2,2018-04-02 00:00:00,60750101001000,NaN,0
3,2018-04-02 12:00:00,60750101001000,NaN,0
4,2018-04-03 00:00:00,60750101001000,NaN,0
5,2018-04-03 12:00:00,60750101001000,NaN,0
6,2018-04-04 00:00:00,60750101001000,NaN,0
7,2018-04-04 12:00:00,60750101001000,NaN,0
8,2018-04-05 00:00:00,60750101001000,NaN,0
9,2018-04-05 12:00:00,60750101001000,NaN,0


In [19]:
encamp_count = []
for index, row in encamp_matrix.iterrows():
    
    encamp_use = encampments[(encampments['Opened_rnd'] < row['Opened_rnd'] + datetime.timedelta(days=10))&
                            (encampments['Opened_rnd'] > row['Opened_rnd'] - datetime.timedelta(days=10))]
    adder = 0
    for index2, row2 in encamp_use.iterrows():
        
        if row['poly'].contains(Point(row2['Longitude'],row2['Latitude'])):
            adder += 1
    encamp_count.append(adder)

In [24]:
#encamp_matrix['encamp_count'] = encamp_count
encamp_matrix = encamp_matrix.loc[:,[
    
    'block_fips',
    'Opened_rnd',
    'encamp_count'
]]
encamp_matrix['Opened_rnd'] = test['Opened_rnd']
encamp_matrix.to_csv('..//..//data//created//encamp_feature.csv', index=False)

In [27]:
encamp_matrix['Opened_rnd'].max()

Timestamp('2018-05-01 00:00:00')